In [14]:
# ! pip install xmlschema
# ! pip install scikit-learn
# ! pip install openai
! pip install tiktoken
# ! pip install trl
# ! pip install lzma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.2 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
! python3 --version

Python 3.10.12


In [7]:

import os
import gc

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer

import json
import wandb
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
# Print current working directory
# !ls -alh /var/model/Phind-CodeLlama-34B-v2
# Change to /var/model/Phind-CodeLlama-34B-v2
# os.chdir( "/var/model/Phind-CodeLlama-34B-v2" )
# Print current working directory
# os.getcwd()
! ls -alh /var/model/

total 404K
drwxr--r-- 32 1001 1001 4.0K Jan  2 15:00 .
drwxr-xr-x  1 root root 4.0K Jan  2 16:36 ..
drwxr--r--  3 1001 1001 4.0K Aug 22 13:57 .idea
drwxr--r-- 16 1001 1001 4.0K Aug 15 19:52 Auto-GPT
drwxrwxr-x  6 1001 1001 4.0K Nov 19 02:04 CodeLlama-13b-Instruct-hf
drwxrwxr-x  7 1001 1001 4.0K Dec 19 14:10 CodeLlama-7b-Instruct-hf
-rwxr--r--  1 1001 1001 3.3K Feb 28  2023 Dockerfile
drwxrwxr-x 15 1001 1001 4.0K Nov  8 18:06 OpenLLM
drwxrwxr-x  6 1001 1001 4.0K Jan  2 15:54 Phind-CodeLlama-34B-v2
drwxrwxr-x 13 1001 1001 4.0K Dec 13 00:08 TTS
drwxr--r--  6 1001 1001 4.0K Feb 28  2023 ampe
drwxr--r--  5 1001 1001 4.0K Jul  5 15:19 cache
-rwxr--r--  1 1001 1001 254K Aug  1 01:17 chat_history.txt
drwxr--r--  5 1001 1001 4.0K Apr 28  2023 coursework
drwxrwxr-x  8 1001 1001 4.0K Nov  3 15:33 cursor-flask-chatbot
drwxr--r--  6 1001 1001 4.0K Sep  3 23:49 cursor-flask-js-websocket
drwxr-xr-x  3 1001 1001 4.0K Oct 16 18:49 cursor-gib
drwxr--r--  3 1001 1001 4.0K Sep  6 19:51 cursor-langchain-ex

In [3]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
%env WANDB_PROJECT=Phind-CodeLlama-34B-v2-peft-fine-tuning

env: WANDB_PROJECT=Phind-CodeLlama-34B-v2-peft-fine-tuning


In [5]:
# dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
# split = "train[:10%]"
# # finetunes_model_name = "output/codellama-7b-finetuned-int4-python-18k-alpaca"

In [6]:
# from datasets import load_dataset
# 
# dataset = load_dataset( dataset_name, split=split )

In [7]:
# dataset[ 0 ]

In [21]:
from xmlschema import XMLSchema

os.chdir( "/var/model/genie-in-the-box/src" )
import lib.utils.util     as du
import lib.utils.util_xml as dux
import lib.utils.util_pytorch as dupt

from ephemera.prompts.xml_fine_tuning_prompt_generator import XmlFineTuningPromptGenerator


## Get training dataset

In [10]:
path = "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-train.jsonl"
deepily_dataset_train = du.get_file_as_list( path )
deepily_dataset_train = [ json.loads( line ) for line in deepily_dataset_train ]
# deepily_dataset_train

In [11]:
path = "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-test.jsonl"
deepily_dataset_test = du.get_file_as_list( path )
deepily_dataset_test = [ json.loads( line ) for line in deepily_dataset_test ]
# deepily_dataset_test

In [13]:
len( deepily_dataset_test )

100

In [6]:
# Use the Task below and the Input given to write the Response, which is a programmatic instruction that can solve the following Task:
def prompt_instruction_format( sample ):
    
  return f"""### Instruction:
    Use the Task below and the Input given to write a Response that can solve the following Task:

    ### Task:
    {sample['instruction']}

    ### Input:
    {sample['input']}

    ### Response:
    {sample['output']}
    """

In [15]:
for line in prompt_instruction_format( deepily_dataset_test[ 0 ] ).split( "\n" ): print( line )

### Instruction:
    Use the Task below and the Input given to write a Response that can solve the following Task:

    ### Task:
    Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

        You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

        Requirement: You MUST NOT use python code to answer this question.
        Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
        Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

    ### Input:
    
        Below is the raw human voice c

In [8]:
def generate_text( foo_tokenizer, model, question, max_new_tokens=256, debug=False ):
    
    instruction = f"""### Instruction:
    Use the Task below and the Input given to write a Response that can solve the following Task:

    ### Task:
    Your job is to discern the intent of a human voice command transcription and translate it into a standardized command that a browser on your computer would understand.

    You will be given a human voice command and a list of possible standardized commands. You must choose the correct standardized command from the following list: `'search new tab', 'search current tab', 'search google new tab', 'search google current tab', 'search google scholar new tab', 'search google scholar current tab' and 'none'`.

    Requirement: You MUST NOT use python code to answer this question.
    Requirement: You MUST use your linguistic knowledge and intuition to answer this question.
    Hint: Anything that isn't a part of the command itself should be treated as arguments related to the command.

    ### Input:
    
    Below is the raw human voice command transcription formatted using simple XML:
    
    <human>
        <voice-command>{question}</voice-command>
    </human>
    
    The standardized command that you translate MUST be returned wrapped in simple, well-formed XML:
    
    <response>
        <browser-command></browser-command>
        <args></args>
    </response>

    Requirement: The first word of your response MUST be `<response>`

    ### Response:"""
    
    
    device = "cuda:0"
    inputs = foo_tokenizer( instruction, return_tensors="pt" ).to( device )
    
    stop_token_id = foo_tokenizer.encode( "</response>" )[ 0 ]
    
    generation_output = model.generate(
        input_ids=inputs[ "input_ids" ],
        attention_mask=inputs[ "attention_mask" ],
        max_new_tokens=max_new_tokens,
        eos_token_id=stop_token_id,
        pad_token_id=stop_token_id
    )
        
    if debug: 
        print( "generation_output[ 0 ]:", generation_output[ 0 ], end="\n\n" )
        print( "generation_output[ 0 ].shape:", generation_output[ 0 ].shape, end="\n\n" )
    
    # Skip decoding the prompt part of the output   
    input_length = inputs[ "input_ids" ].size( 1 )
    raw_output = foo_tokenizer.decode( generation_output[ 0 ][ input_length: ] )
    
    if debug: 
        print( "raw_output:", raw_output, end="\n\n" )
        print(  "len( raw_output ):", len( raw_output ), end="\n\n")
    
    # response   = raw_output.split( "### Response:" )[ 1 ]
    
    response = raw_output.replace( "</s><s>", "" ).strip()
    
    return response

question = "Ask Google scholar about QLORA and PEFT fine-tuning for XML output, show results in Another tab"

# for line in generate_text( tokenizer, base_model, question ).split( "\n" ): print( line )

In [25]:
import os 
os.chdir( "/var/model/Phind-CodeLlama-34B-v2" )

In [18]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
print( bnb_config )
tokenizer              = AutoTokenizer.from_pretrained( "." )
tokenizer.pad_token    = tokenizer.eos_token
tokenizer.padding_side = "right"

# ¡OJO! Why are All the examples I'm finding online turning off the cache here? It makes a huge performance difference: 21 vs 14 tokens per second!
# Now I remember: cashing is suspended while training, should only be used for inference
base_model = AutoModelForCausalLM.from_pretrained(
    ".", quantization_config=bnb_config, device_map="auto", low_cpu_mem_usage=True, use_cache=True, use_flash_attention_2=True
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


BitsAndBytesConfig {
  "bnb_4bit_compute_dtype": "bfloat16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
# for line in generate_text( tokenizer, base_model, product ).split( "\n" ): print( line )

In [19]:
# def ppl_model( model, tokenizer, dataset ):
#     nlls = [ ]
#     max_length = 4096 #2048
#     stride = 512
#     for s in tqdm( range( len( dataset[ 'prompt' ] ) ) ):
#         encodings = tokenizer( dataset[ 'prompt' ][ s ], return_tensors="pt", truncation=True, max_length=max_length )
#         seq_len = encodings.input_ids.size( 1 )
#         prev_end_loc = 0
#         for begin_loc in range( 0, seq_len, stride ):
#             end_loc = min( begin_loc + max_length, seq_len )
#             trg_len = end_loc - prev_end_loc
#             input_ids = encodings.input_ids[ :, begin_loc:end_loc ].to( "cuda" )
#             target_ids = input_ids.clone()
#             target_ids[ :, :-trg_len ] = -100
#             with torch.no_grad():
#                 outputs = model( input_ids, labels=target_ids )
#                 neg_log_likelihood = outputs.loss
#             nlls.append( neg_log_likelihood )
#             prev_end_loc = end_loc
#             if end_loc == seq_len:
#                 break
#     ppl = torch.exp( torch.stack( nlls ).mean() )
#     return ppl.item()
# 
# # ppl for "iamtarun/python_code_instructions_18k_alpaca": 4.75
# # ppl for "deepily-xml-prompts": 

In [20]:
# ppl_model( base_model, tokenizer, dataset )

# This error happens whenever the training object is instantiated twice in a session. There's got to be a better way to know whatever actions the
# training object has taken without having to restart The notebook server
# RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument tensors in method wrapper_CUDA_cat)

In [21]:
import gc
# del base_model
torch.cuda.empty_cache()
gc.collect()

1496

In [22]:
base_model.hf_device_map

{'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.3': 0,
 'model.layers.4': 0,
 'model.layers.5': 0,
 'model.layers.6': 0,
 'model.layers.7': 0,
 'model.layers.8': 0,
 'model.layers.9': 0,
 'model.layers.10': 0,
 'model.layers.11': 0,
 'model.layers.12': 0,
 'model.layers.13': 0,
 'model.layers.14': 0,
 'model.layers.15': 0,
 'model.layers.16': 0,
 'model.layers.17': 0,
 'model.layers.18': 0,
 'model.layers.19': 0,
 'model.layers.20': 0,
 'model.layers.21': 1,
 'model.layers.22': 1,
 'model.layers.23': 1,
 'model.layers.24': 1,
 'model.layers.25': 1,
 'model.layers.26': 1,
 'model.layers.27': 1,
 'model.layers.28': 1,
 'model.layers.29': 1,
 'model.layers.30': 1,
 'model.layers.31': 1,
 'model.layers.32': 1,
 'model.layers.33': 1,
 'model.layers.34': 1,
 'model.layers.35': 1,
 'model.layers.36': 1,
 'model.layers.37': 1,
 'model.layers.38': 1,
 'model.layers.39': 1,
 'model.layers.40': 1,
 'model.layers.41': 1,
 'model.layers.42'

In [23]:
for name, param in base_model.named_parameters():
    print(f"Parameter {name} is on device {param.device}")

Parameter model.embed_tokens.weight is on device cuda:0
Parameter model.layers.0.self_attn.q_proj.weight is on device cuda:0
Parameter model.layers.0.self_attn.k_proj.weight is on device cuda:0
Parameter model.layers.0.self_attn.v_proj.weight is on device cuda:0
Parameter model.layers.0.self_attn.o_proj.weight is on device cuda:0
Parameter model.layers.0.mlp.gate_proj.weight is on device cuda:0
Parameter model.layers.0.mlp.up_proj.weight is on device cuda:0
Parameter model.layers.0.mlp.down_proj.weight is on device cuda:0
Parameter model.layers.0.input_layernorm.weight is on device cuda:0
Parameter model.layers.0.post_attention_layernorm.weight is on device cuda:0
Parameter model.layers.1.self_attn.q_proj.weight is on device cuda:0
Parameter model.layers.1.self_attn.k_proj.weight is on device cuda:0
Parameter model.layers.1.self_attn.v_proj.weight is on device cuda:0
Parameter model.layers.1.self_attn.o_proj.weight is on device cuda:0
Parameter model.layers.1.mlp.gate_proj.weight is on

## Set up training arguments

In [15]:
from peft import LoraConfig, get_peft_config, PeftModel, PeftConfig, get_peft_model, AutoPeftModelForCausalLM

peft_config = LoraConfig(
    r=16, 
    lora_alpha=32, 
    # When target_modules was disabled, it was causing detention layers to be assigned to the CPU, throwing this runtime error:
    # RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! 
    # (when checking argument for argument mat2 in method wrapper_CUDA_mm)
    target_modules=[ "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj" ], 
    lora_dropout=0.10, 
    bias="none", 
    task_type="CAUSAL_LM"
)

In [21]:
del trainingArgs
del trainer
import gc
torch.cuda.empty_cache()
gc.collect()

1878

In [16]:
# Define the training arguments
trainingArgs = TrainingArguments(
    output_dir="./training-results", # Output directory where the model predictions and checkpoints will be stored
    num_train_epochs=4, # Number of training epochs
    per_device_train_batch_size=1, # Batch size per GPU for training
    gradient_accumulation_steps=4,  # Number of update steps to accumulate the gradients for
    gradient_checkpointing=True,# Enable gradient checkpointing
    optim="paged_adamw_32bit", # Optimizer to use
    #save_steps=save_steps,
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.001,
    
    # Setting this may help with the warning message: The input hidden states seems to be silently casted in float32, 
    # this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.
    fp16=True,
    # Test to confirm that this works!
    # BTW: according to PHIND, this may actually improve fine-tuning performance as well: https://www.phind.com/search?cache=ygn9dbyl0ij4kotmgns2nsrw
    
    bf16=False,
    # tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    #max_steps=max_steps,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    report_to="wandb",
    seed=42
)
# Create the trainer
trainer = SFTTrainer(
    model=base_model,
    train_dataset=deepily_dataset_train,
    eval_dataset=deepily_dataset_test,
    peft_config=peft_config,
    max_seq_length=4096, #2048,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)

In [24]:
def print_trainable_parameters( model ):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [25]:
print_trainable_parameters( base_model )

trainable params: 525,082,624 || all params: 17,134,526,464 || trainable%: 3.06


In [26]:
for name, param in base_model.named_parameters():
    print(f"Parameter {name} is on {param.device}")


Parameter model.embed_tokens.weight is on cuda:0
Parameter model.layers.0.self_attn.q_proj.weight is on cuda:0
Parameter model.layers.0.self_attn.k_proj.weight is on cuda:0
Parameter model.layers.0.self_attn.v_proj.weight is on cuda:0
Parameter model.layers.0.self_attn.o_proj.weight is on cuda:0
Parameter model.layers.0.mlp.gate_proj.weight is on cuda:0
Parameter model.layers.0.mlp.up_proj.weight is on cuda:0
Parameter model.layers.0.mlp.down_proj.weight is on cuda:0
Parameter model.layers.0.input_layernorm.weight is on cuda:0
Parameter model.layers.0.post_attention_layernorm.weight is on cuda:0
Parameter model.layers.1.self_attn.q_proj.weight is on cuda:0
Parameter model.layers.1.self_attn.k_proj.weight is on cuda:0
Parameter model.layers.1.self_attn.v_proj.weight is on cuda:0
Parameter model.layers.1.self_attn.o_proj.weight is on cuda:0
Parameter model.layers.1.mlp.gate_proj.weight is on cuda:0
Parameter model.layers.1.mlp.up_proj.weight is on cuda:0
Parameter model.layers.1.mlp.down

In [20]:
trainer.train()

#stop reporting to wandb
wandb.finish()

# save model
trainer.save_model()

print( "Model saved" )

# {'loss': 0.5582, 'learning_rate': 1.1111111111111112e-05, 'epoch': 0.03}
# {'loss': 0.512, 'learning_rate': 2.2222222222222223e-05, 'epoch': 0.05}
# {'loss': 0.4509, 'learning_rate': 3.3333333333333335e-05, 'epoch': 0.07}
# {'loss': 0.3753, 'learning_rate': 4.4444444444444447e-05, 'epoch': 1.0}
# {'loss': 0.3123, 'learning_rate': 5.555555555555556e-05, 'epoch': 1.03}
# {'loss': 0.259, 'learning_rate': 6.666666666666667e-05, 'epoch': 1.05}
# {'loss': 0.1952, 'learning_rate': 7.777777777777778e-05, 'epoch': 1.08}
# {'loss': 0.1454, 'learning_rate': 8.888888888888889e-05, 'epoch': 2.01}
# {'loss': 0.1129, 'learning_rate': 0.0001, 'epoch': 2.03}
# {'loss': 0.0995, 'learning_rate': 0.00011111111111111112, 'epoch': 2.06}
# {'loss': 0.0877, 'learning_rate': 0.00012222222222222224, 'epoch': 2.08}
# {'loss': 0.0815, 'learning_rate': 0.00013333333333333334, 'epoch': 3.01}

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.


{'loss': 0.5477, 'learning_rate': 4.166666666666667e-05, 'epoch': 0.03}
{'loss': 0.4307, 'learning_rate': 8.333333333333334e-05, 'epoch': 0.05}
{'loss': 0.3226, 'learning_rate': 0.000125, 'epoch': 0.07}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.2261, 'learning_rate': 0.0001666666666666667, 'epoch': 1.0}
{'loss': 0.1358, 'learning_rate': 0.00019999918050612108, 'epoch': 1.03}
{'loss': 0.0998, 'learning_rate': 0.0001999704996306308, 'epoch': 1.05}
{'loss': 0.0897, 'learning_rate': 0.00019990085749160822, 'epoch': 1.08}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.0835, 'learning_rate': 0.00019979028262377118, 'epoch': 2.01}
{'loss': 0.07, 'learning_rate': 0.00019963882033334826, 'epoch': 2.03}
{'loss': 0.0636, 'learning_rate': 0.00019944653267951504, 'epoch': 2.06}
{'loss': 0.0584, 'learning_rate': 0.00019921349844896654, 'epoch': 2.08}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.0562, 'learning_rate': 0.00019893981312363562, 'epoch': 3.01}
{'loss': 0.0463, 'learning_rate': 0.00019862558884157068, 'epoch': 3.04}
{'loss': 0.0459, 'learning_rate': 0.00019827095435098925, 'epoch': 3.06}
{'loss': 0.048, 'learning_rate': 0.00019787605495752528, 'epoch': 3.09}
{'train_runtime': 6304.2352, 'train_samples_per_second': 0.508, 'train_steps_per_second': 0.127, 'train_loss': 0.15215845870507227, 'epoch': 3.1}


train/epoch,▁▁▁▃▃▃▃▆▆▆▆█████
train/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train/learning_rate,▁▃▅▇███████████
train/loss,█▆▅▄▂▂▂▂▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,3.1
train/global_step,77


Model saved


In [19]:
# wandb.finish()

In [28]:
for line in generate_text( tokenizer, base_model, question ).split( "\n" ): print( line )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


generation_output[ 0 ]: tensor([    1,   835,  2799,  4080, 29901,    13,  1678,  4803,   278,  9330,
         2400,   322,   278, 10567,  2183,   304,  2436,   263, 13291,   393,
          508,  4505,   278,  1494,  9330, 29901,    13,    13,  1678,   835,
         9330, 29901,    13,  1678,  3575,  4982,   338,   304,  2313,   824,
          278,  7609,   310,   263,  5199,  7314,  1899,  1301,  3395,   322,
        14240,   372,   964,   263,  3918,  1891,  1899,   393,   263,  4714,
          373,   596,  6601,   723,  2274, 29889,    13,    13,  1678,   887,
          674,   367,  2183,   263,  5199,  7314,  1899,   322,   263,  1051,
          310,  1950,  3918,  1891,  8260, 29889,   887,  1818,  6755,   278,
         1959,  3918,  1891,  1899,   515,   278,  1494,  1051, 29901, 16218,
         4478,   716,  4434,   742,   525,  4478,  1857,  4434,   742,   525,
         4478,  5386,   716,  4434,   742,   525,  4478,  5386,  1857,  4434,
          742,   525,  4478,  5386, 2134

In [47]:
# Drops 16.4/19.0 GB per GPU down to 3.25 GB per GPU!
import gc
base_model = None 
adapter_plus_model = None
torch.cuda.empty_cache() 
gc.collect()

493

In [32]:
os.chdir( "/var/model/Phind-CodeLlama-34B-v2")

In [33]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
print( bnb_config )
tokenizer              = AutoTokenizer.from_pretrained( "." )
tokenizer.pad_token    = tokenizer.eos_token
tokenizer.padding_side = "right"

# ¡OJO! Why were we turning off the cash here? 
# We're not! It makes a huge performance difference: 21 vs 14 tokens per second!
base_model = AutoModelForCausalLM.from_pretrained(
    ".", quantization_config=bnb_config, device_map="auto", low_cpu_mem_usage=True, use_cache=True, use_flash_attention_2=True
)

BitsAndBytesConfig {
  "bnb_4bit_compute_dtype": "bfloat16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [34]:
dupt.print_device_allocation( base_model )


model.embed_tokens.weight: cuda:0
model.layers.0.self_attn.q_proj.lora_A.default.weight: cuda:0
model.layers.0.self_attn.q_proj.lora_B.default.weight: cuda:0
model.layers.0.self_attn.q_proj.base_layer.weight: cuda:0
model.layers.0.self_attn.k_proj.lora_A.default.weight: cuda:0
model.layers.0.self_attn.k_proj.lora_B.default.weight: cuda:0
model.layers.0.self_attn.k_proj.base_layer.weight: cuda:0
model.layers.0.self_attn.v_proj.lora_A.default.weight: cuda:0
model.layers.0.self_attn.v_proj.lora_B.default.weight: cuda:0
model.layers.0.self_attn.v_proj.base_layer.weight: cuda:0
model.layers.0.self_attn.o_proj.lora_A.default.weight: cuda:0
model.layers.0.self_attn.o_proj.lora_B.default.weight: cuda:0
model.layers.0.self_attn.o_proj.base_layer.weight: cuda:0
model.layers.0.mlp.gate_proj.lora_A.default.weight: cuda:0
model.layers.0.mlp.gate_proj.lora_B.default.weight: cuda:0
model.layers.0.mlp.gate_proj.base_layer.weight: cuda:0
model.layers.0.mlp.up_proj.lora_A.default.weight: cuda:0
model.la

In [35]:
response = generate_text( tokenizer, base_model, question )
response = f"[{response}]"
for line in response.split( "\n" ): print( line )

[<response>
            <browser-command>search google scholar new tab</browser-command>
            <args>QLORA and PEFT fine-tuning for XML output</args>
        </response>]


In [36]:
from peft import PeftModel
adapter_plus_model = PeftModel.from_pretrained( base_model, "adapters/00-browser-vox-command", use_flash_attention_2=True )

In [44]:
from accelerate import Accelerator

accelerator = Accelerator()

adapter_plus_model = accelerator.prepare( adapter_plus_model )


In [45]:
dupt.print_device_allocation( adapter_plus_model )

base_model.model.model.embed_tokens.weight: cuda:0
base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: cpu
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: cpu
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight: cuda:0
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: cpu
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: cpu
base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight: cuda:0
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight: cpu
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: cpu
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight: cuda:0
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight: cpu
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight: cpu
base_model.model.model.layers.0.self_attn.o_proj.base_layer.weight: cuda:0
base_model.model.model.layers.0.mlp.gate_

In [43]:
for line in generate_text( tokenizer, adapter_plus_model, question ).split( "\n" ): print( line )

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat2 in method wrapper_CUDA_mm)

In [46]:
deepily_dataset_test[ 0 ].keys()

dict_keys(['instruction', 'input', 'output', 'prompt'])

In [48]:
deepily_dataset_test[ 0 ][ 'input' ]

'\n        Below is the raw human voice command transcription formatted using simple XML:\n        \n        <human>\n            <voice-command>Start a Google search on How do you address resource-related warnings in Python, especially regarding resource usage? in a new tab</voice-command>\n        </human>\n        \n        The standardized command that you translate MUST be returned wrapped in simple, well-formed XML:\n        \n        <response>\n            <browser-command></browser-command>\n            <args></args>\n        </response>\n\n        Requirement: The first word of your response MUST be `<response>`'

In [50]:
import pandas as pd

In [82]:
test_df = pd.read_json( "/var/model/genie-in-the-box/src/ephemera/prompts/data/voice-commands-xml-test.jsonl", lines=True )
test_df.head()

,instruction,input,output,prompt
0,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...
1,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...
2,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...
3,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...
4,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...


In [83]:
questions = test_df[ 'input' ].tolist()
for idx, question in enumerate( questions ):
    question = dux.get_value_by_xml_tag_name( question, "voice-command" )
    # print( question )
    questions[ idx ] = question
    
questions[ 0 ]

'Now in this tab, Google Scholar best fashion trends this year'

In [84]:
generate_text( tokenizer, adapter_plus_model, questions[ 0 ] )

'<response>\n            <browser-command>search google scholar current tab</browser-command>\n            <args>best fashion trends this year</args>\n        </response>'

In [85]:
import re

In [90]:
responses = [ ]
for idx, question in enumerate( questions ):
    # Display progress every nth iteration
    print( f"[{idx + 1}] of [{len( questions )}] [{( ( idx + 1 ) / len( questions ) ) * 100:.1f}%] [{question}]" )
    response = generate_text( tokenizer, adapter_plus_model, question )
    response = re.sub( r'>\s+<', '><', response )
    print( f"[{response}]" )
    responses.append( response )
    

[1] of [1000] [0.1%] [Now in this tab, Google Scholar best fashion trends this year]
[<response><browser-command>search google scholar current tab</browser-command><args>best fashion trends this year</args></response>]
[2] of [1000] [0.2%] [Do a Google Scholar Warning search in a new tab]
[<response><browser-command>search google scholar new tab</browser-command><args>Warning</args></response>]
[3] of [1000] [0.3%] [New tab, lookup for Using Pandas for ETL processes]
[<response><browser-command>search new tab</browser-command><args>Using Pandas for ETL processes</args></response>]
[4] of [1000] [0.4%] [Use this tab to Google AI for social good initiatives: What are some notable AI for social good initiatives?]
[<response><browser-command>search google current tab</browser-command><args>AI for social good initiatives: What are some notable AI for social good initiatives?</args></response>]
[5] of [1000] [0.5%] [In a fresh tab, perform a Google search of How can buffer-related errors be 

In [91]:
len( responses )

1000

In [92]:
test_df[ 'response' ] = responses
test_df.head()

,instruction,input,output,prompt,response
0,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,<response><browser-command>search google schol...
1,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,<response><browser-command>search google schol...
2,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,<response><browser-command>search new tab</bro...
3,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,<response><browser-command>search google curre...
4,Your job is to discern the intent of a human v...,\n Below is the raw human voice command...,\n <response>\n <browser-com...,### Instruction:\n Use the Task and Input g...,<response><browser-command>search google new t...


In [67]:
# Install notebook magic that forces reload a source code
%load_ext autoreload

In [93]:
os.chdir( "/var/model/genie-in-the-box/src" )
os.getcwd()

'/var/model/genie-in-the-box/src'

In [94]:
%autoreload
from ephemera.prompts.xml_fine_tuning_prompt_generator import XmlFineTuningPromptGenerator

# Advice from phind when auto reload fails: https://www.phind.com/search?cache=l7c51x1v3tids6l43mgkkm9j
os.chdir( "/var/model/genie-in-the-box/src" )
%run "ephemera/prompts/xml_fine_tuning_prompt_generator.py"  

xml_ftp_generator = XmlFineTuningPromptGenerator( path_prefix="/var/model/genie-in-the-box" )

test_df = xml_ftp_generator.validate_prompts_and_responses( test_df )

xml_ftp_generator.print_validation_stats( test_df )

Running XmlFineTuningPromptGenerator...
Commands file for [search new tab] exists: True
Commands file for [search current tab] exists: True
Commands file for [search google new tab] exists: True
Commands file for [search google current tab] exists: True
Commands file for [search google scholar new tab] exists: True
Commands file for [search google scholar current tab] exists: True
------------------------------------------------------------------------------------------------------------------------
- Validation Stats
------------------------------------------------------------------------------------------------------------------------

               Is valid xml 100.0%
          Contains response 100.0%
   Contains browser command 100.0%
              Contains args 100.0%
          Response is exact 99.3%
Response has correct values 99.3%
 Browser command is correct 99.6%
            Args is correct 99.7%
